<a href="https://colab.research.google.com/github/fedhere/FASTlab/blob/master/PHYS838SNcosmo/SNeIa_HubbleDiagram_instructions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise  – Fit an Hubble Diagram
Originally from 
https://supernovae.in2p3.fr/sdss_snls_jla/ReadMe.html
Edited by @fedhere for PHYS 838 Galaxies and Cosmology UD 2023

_This notebook is not complete - you need to fill in code in four places for it to work - look for where it says "replace with your code here"_ 

You can check your results against the solution notebook in the repository https://github.com/fedhere/FASTlab/blob/master/PHYS838SNcosmo/SNeIa_HubbleDiagram_solution.ipynb

## The SN Ia Science in short

The Type Ia Supernova event is the thermonuclear runaway of a white dwarf. This bright event is extremely stable and the maximum of luminosity of any explosion reaches roughly the same maximum magnitude `M0`. Two additional parameters: the speed of evolution of the explosion `x1` (also called stretch) and the color of the event `color` enable to further standardize the SN maximum of luminosity. Thanks to the statibility of the absolute SN magnitude, the observation of the effective SN magnitude is a direct indication of the event's distance. Combined with the redshift measurement of the Supernovae -the redshift tracing the overall expansion of the Universe since the SN event- we can measurment the history of the expansion of the Universe. The magnitude vs. redshift plot is called an Hubble Diagram and the Hubble diagram of the SNe Ia enabled the discovery of the accelerated expansion of the Universe in 1999, and thereby of the existance of dark energy (Noble 2011)

## Data you have to measures the density Dark Energy

The latest compilation of Type Ia Supernovae Data has ~1000 SN Ia distance measurements. The data are registered
in 

`jla_lcparams.txt`

## The Model

In this example we will use the most straightforward analysis. The stretch and color corrected magnitude of the SNe Ia is:

$$
mb_{corr} = mb - (x1 \times \alpha - color \times \beta)
$$

The expected magnitude of the SNe Ia is:

$$
mb_{expected} = \mu \left(z, \Omega \right) + M_0
$$

where $\mu\left(z, \Omega \right)$ is the distance modulus (distance in log scale) for a given cosmology. To have access to $\mu$ use `astropy`. 

In the flat Lambda CDM model, the only free cosmological parameter you can fit is `Omega_m`, the density of matter today, knowing that, in that case, the density of dark energy is `1-Omega_m`

#### Astropy

The website with all the information is here http://www.astropy.org/

On colab you can access astropy by typing 
`pip install astropy` 

To get the cosmology for an Hubble Constant of 70 (use that) and for a given density of matter (Omega_m, which will be one of your free parameter):
```python
from astropy import cosmology
cosmo = cosmology.FlatLambdaCDM(70, Omega_m)
```
To get the distance modulus
```python
from astropy import cosmology
mu = cosmology.FlatLambdaCDM(70, Omega_m).distmod(z).value
```




## Your Job: find the density of Dark energy

Create a Chi2 fitting class that enables to derive the density of dark energy. You should find `Omega_m~0.3` so a universe composed of ~70% of dark energy and 30% of matter

Plot an Hubble diagram showing the corrected SN mangitude (`mb_corr`) as a function of the redshift (`zcmb`). Overplot the model. 

**Remark** `alpha` `beta` and `M0` have to be fitted together with the cosmology, we call that 'nuisance parameters'

**Remark 2** ignore errors on the redshift (`z`), but take into account errors on `mb` and on the `x1` and `color` parameters. For this example ignore the covariance terms.

***
# Correction 
***

In [ ]:
import warnings
# No annoying warnings
warnings.filterwarnings('ignore')
# Because we always need that
# plot within the notebook
# %matplotlib inline dont need on colab
import numpy as np
import matplotlib.pyplot as mpl

## First Steps access the Data

#### Import for the cosmological analysis (an also the convenient table)

In [ ]:
from astropy import table, cosmology

#### Read the data

In [ ]:
data = table.Table.read("https://raw.githubusercontent.com/fedhere/FASTlab/master/PHYS838SNcosmo/jla_lcparams.txt", format="ascii")

In [ ]:
data

In [ ]:
data.colnames

For each SN there is a set of values that are fit to the lightcurve. This is a highly dimenz=sional space ofr a fit! (note: normally tmax date of maximum is also fit...)

*name*: name of the SN, NOT A PARAMETER

*zcmb*: CMB frame redshift (including peculiar velocity corrections for nearby supernova based on the models of M.J. Hudson)

_zhel_: Heliocentric redshift (note both zcmb and zhel are needed to compute the luminosity distance)

_dz_: redshift error ALL d-quantities are errors on fit quantities

_mb_: B band peak magnitude

_dmb_: Error in mb (includes contributions from intrinsic dispersion, lensing, and redshift uncertainty)

_x1_: SALT2 shape parameter 
_dx1_: Error in shape parameter

_colour_: Colour parameter
_dcolour_: Error in colour

_3rdvar_: In these files, the log_10 host stellar mass
_d3rdvar_: Error in 3rdvar

*cov_m_s*: The covariance between mb and x1

*cov_m_c*: The covariance between mb and colour

*cov_s_c*: The covariance between x1 and colour

*set*: A number indicating which sample this SN belongs to, NOT A PARAMETER with

     1 - SNLS, 
     2 - SDSS, 
     3 - low-z, 
     4 - Riess HST

_ra_: Right Ascension in degree (J2000), NOT A PARAMETER

_dec_: Declination in degree (J2000), NOT A PARAMETER

_biascor_: The correction for analysis bias applied to measured magnitudes (this correction is already applied to mb, original measurements can be obtained by subtracting this term to mb), NOT A PARAMETER

## The Chi2 Method

an introduction of the "property" and "setter" decorators

decorators are kind-of functions of function.
 
Check e.g., http://thecodeship.com/patterns/guide-to-python-function-decorators/

In [ ]:
# copy paste of a class build before

class Chi2Fit( object ):
    
    def __init__(self, jla_data=None):
        """ init the class
        
        Parameters:
        -----------
        jla_data: [astropy.table]
            Astropy Table containing the Supernovae properties
            (zcmb, mb, x1, color etc.)
        
        Return
        ------
        Void
        """
        if jla_data is not None:
            self.set_data(jla_data)

    # ------------------- #
    #   Setter            #
    # ------------------- #        
    def set_data(self, datatable):
        """ Set the data of the class. This must be an astropy table """
        # use this tricks to forbid the user to overwrite self.data...
        self._data = datatable

    def setup(self, parameters):
        """ Set the parameters of the class: 
        - alpha #
        - beta #
        - Om # Omega Matter
        - M0 # 
        """
        self._parameters = parameters  
        
    # ------------------- #
    #   GETTER            #
    # ------------------- #
    def get_mbcorr(self, parameters):
        """ corrected sn magnitude with its associated variance"""
        return self.mb - (self.x1*parameters[0] + self.color*parameters[1]),\
            self.mb_err**2 + (self.x1*parameters[0])**2 + (self.color*parameters[1])**2
    
    def get_mbexp(self, parameters, z=None):
        """ corrected sn magnitude with its associated error"""
        zused = z if z is not None else self.zcmb
        return cosmology.FlatLambdaCDM(70, parameters[2]).distmod(zused ).value + parameters[3]
        
    
    def fit(self, guess):
        """ fit the model to the data 
        
        The methods uses scipy.optimize.minize to fit the model 
        to the data. The fit output is saved as self.fitout, the 
        best fit parameters being self.fitout["x"]
        
        Parameters
        ----------
        guess: [array]
            initial guess for the minimizer. It's size must correspond
            to the amount of free parameters of the model.
            
        Return
        ------
        Void (create self.fitout)
        """
        from scipy.optimize import minimize
        bounds = [[None, None], [None, None], [0, None], [None, None]]
        """CAN YOU FIGURE OUT HOW TO USE scipy.minimize() TO GET THE BEST VALUES?"""
        ... #replace with your code
        
        self._bestfitparameters = self.fitout["x"]
 
    def chi2(self,parameters):
        """ The chi2 of the model with the given `parameters` 
        in comparison to the object's data 
        
        Return
        ------
        float (the chi2)
        """
        """ KNOWING chi2 is Sum_i(model_i - data_i)^2 / variance_i
        AND THAT THE FUNCTION self.get_mbcorr(parameters) 
        GIVES YOU THE MODEL AND THE VARIANCE YOU NEED, 
        WHAT IS THE FUNCTION YOU WANT OT MINIMIZE? 
        SET THE RETURN VALUE OF THIS FUNCTION TO IT"""

        ... #replace with your code
        
        return np.sum(chi2)
    
    def plot(self, parameters):
        """ Vizualize the data and the model for the given
        parameters
        
        Return
        ------
        Void
        """
        fig = mpl.figure()
        ax = fig.add_subplot(1,1,1)
        
        self.setup(parameters)
        set_colors = {
            1: "SteelBlue",
            2: "green",
            3: "IndianRed",
            4: "gold",
        }
        set_names = {
            1:"SNLS", 
            2:"SDSS", 
            3:"low-z", 
            4:"Riess HST"
        }
        for eachset in set(self.sample):
      
            ax.errorbar(self.zcmb[self.sample == eachset], self.setted_mbcorr[self.sample == eachset], # plotting corrected hubble residuals
                    yerr=self.setted_mbcorr_err[self.sample == eachset], 
                    ls="None", marker='o', color=set_colors[eachset], alpha=0.5,
                    ecolor="0.7", label=set_names[eachset])
        x = np.linspace(0.001,1.4,10000)
            #print self.get_cosmo_distmod(parameters,x)
        ax.plot(x, self.get_mbexp(self._parameters, x), '--r', scalex=False, scaley=False)
        """ WHAT ARE THE RIGHT LABELS?"""
        ax.set_xlabel... #replace with your code
        ax.set_ylabel... #replace with your code

        mpl.legend()
        fig.show()

        # LOG X
        fig = mpl.figure()
        ax = fig.add_subplot(1,1,1)
        
        self.setup(parameters)
        
        for eachset in set(self.sample):
      
            ax.errorbar(self.zcmb[self.sample == eachset], self.setted_mbcorr[self.sample == eachset], # plotting corrected hubble residuals
                    yerr=self.setted_mbcorr_err[self.sample == eachset], 
                    ls="None", marker='o', color=set_colors[eachset], alpha=0.5,
                    ecolor="0.7", label=set_names[eachset])
        x = np.linspace(0.001,1.4,10000)
            #print self.get_cosmo_distmod(parameters,x)
        ax.plot(x, self.get_mbexp(self._parameters, x), '--r', scalex=False, scaley=False)
        """ WHAT ARE THE RIGHT LABELS?"""
        ax.set_xlabel("z (CMB)")
        ax.set_ylabel("distance modulus")
        ax.set_xscale('log')
        fig.show()
        
        
    # ================== #
    #  Properties        #
    # ================== #
    @property
    def data(self):
        """ Data table containing the data of the instance """
        return self._data

    @data.setter
    def data(self, newdata):
        """ Set the data """
        # add all the relevant tests
        print("You have set new data")
        self._data = newdata
    
    @property
    def npoints(self):
        """ number of data points """
        return len(self.data)
    
    # ----------
    # - Parameters
    
    @property
    def parameters(self):
        """ Current parameters of the fit """
        if not self.has_parameters():
            raise ValueError("No Parameters defined. See the self.setup() method")
        return self._parameters
    
    def has_parameters(self):
        return "_parameters" in dir(self)
    
    
    
    
    # -- Current Param prop
    @property
    def _alpha(self):
        return self._parameters[0]
    @property
    def _beta(self):
        return self._parameters[1]
    @property
    def _omegam(self):
        return self._parameters[2]
    @property
    def _M0(self):
        return self._parameters[3]
    
    # -------
    # -- Param derived properties
    @property
    def setted_mbcorr(self):
        """ corrected hubble residuals"""
        return self.get_mbcorr(self.parameters)[0]
    @property
    def setted_mbcorr_err(self):
        """ corrected hubble residuals"""
        return np.sqrt(self.get_mbcorr(self.parameters)[1])
    
    @property
    def setted_mu(self):
        """ distance modulus for the given cosmology """
        return cosmology.FlatLambdaCDM(70, _omegam).distmod(self.zcmb).value
    
    @property
    def setted_M0(self):
        """ absolute SN magnitude for the setted parameters """
        return self._M0
    
    @property
    def setted_mbexp(self):
        return self.setted_mu + self._M0
    # -------
    # -- Data derived properties
    @property
    def mb(self):
        """ observed magnitude (in the b-band) of the Supernovae """
        return self.data["mb"]
    
    @property
    def mb_err(self):
        """ observed magnitude (in the b-band) of the Supernovae """
        return self.data["dmb"]
    
    @property
    def x1(self):
        """ Lightcurve stretch """
        return self.data["x1"]
    
    @property
    def x1_err(self):
        """ errors on the Lightcurve stretch """
        return self.data["dx1"]
        
    @property
    def color(self):
        """ Lightcurve color """
        return self.data["color"]
    
    @property
    def color_err(self):
        """ errors on the Lightcurve color """
        return self.data["dcolor"]
    
    @property
    def zcmb(self):
        """ cosmological redshift of the Supenovae """
        return self.data["zcmb"]

    @property
    def sample(self):
        """ observed magnitude (in the b-band) of the Supernovae """
        return self.data["set"]
    
    
    

In [ ]:
c = Chi2Fit(data)

In [ ]:
alphapar = -0.18980149
betapar = 3.65435315
OmegaM = 0.32575054
MagZP = -19.06810566

c.setup([ alphapar, betapar, OmegaM, MagZP])

In [ ]:
c.data

In [ ]:
c.setted_mbcorr # corrected apparent magnitude

In [ ]:
print("""Which SN samples are used?
     1 - SNLS, 
     2 - SDSS, 
     3 - low-z, 
     4 - Riess HST
      """)
set(c.data["set"])

In [ ]:
c.setted_M0 # Absolute Magnitude

# FIT

In [ ]:
c.fit([0.13, 3, 0.2, -20]) # passing some initial parameters to the function that are not so great

ORIGINAL RESULT
```
message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 317.79735399748114
        x: [-1.898e-01  3.654e+00  3.259e-01 -1.907e+01]
      nit: 33
      jac: [-4.150e-04  2.842e-05  0.000e+00 -1.177e-03]
     nfev: 285
     njev: 57


```



In [ ]:
print("BEST FIT PARAMS")
print("""originally: 
alphapar = -0.18980149
betapar = 3.65435315
OmegaM = 0.32575054
MagZP = -19.06810566
""")

print("""initial guess:
      
      0.13, 
      3, 
      0.2, 
      -20""")

print("best fit parameters", c._bestfitparameters)


ORIGINAL RESULT

```
BEST FIT PARAMS
originally: 
alphapar = -0.18980149
betapar = 3.65435315
OmegaM = 0.32575054
MagZP = -19.06810566

initial guess:
      
      0.13, 
      3, 
      0.2, 
      -20
best fit parameters [ -0.18980153   3.65434904   0.32590185 -19.0680978 ]

```



In [ ]:
c.plot(c._bestfitparameters)

# COMPARE WITH 

original nb https://supernovae.in2p3.fr/sdss_snls_jla/ReadMe.html

see also https://notebook.community/EliseJ/astroABC/examples/supernovae foran MCMC solution

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cd5d88a9-4330-4aa9-8029-ed5e4d98d510' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>